In [404]:
%reset -f

In [405]:
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog

img_file = '2.1.01.tiff'

intensities = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
unique, counts = np.unique(intensities, return_counts=True)

height = np.shape(intensities)[0]
width = np.shape(intensities)[1]

u = unique.copy()
c = counts.copy()
for i in range(0,unique[-1]+1):
    if u[i] != i:
        u = np.insert(u,i,i)
        c = np.insert(c,i,0)

z = np.zeros(255-unique[-1])
all_counts = np.append(c,z)
a = np.array(range(unique[-1]+1, 256))
var = np.append(u,a)

occurance_sum = sum(all_counts)
prbs = all_counts/occurance_sum
# plt.plot(var, all_counts)
plt.plot(var, prbs)

IndexError: tuple index out of range

In [394]:
# print(intensities)
cv2.imshow('orig_image',intensities)
cv2.waitKey(1000)
filename = '1.1.webp'
cv2.imwrite(filename, intensities)
cv2.waitKey(1000)

-1

In [395]:
cv2.destroyAllWindows()

In [396]:
cells_num = 256

all_yijs = []
all_dijs = []
all_prb0s = []
for i in range(0,cells_num-1):
    for j in range(i+1,cells_num):
# Strategy1: Eve knows the source statistics which is rare in image encrytion
#         possible_yij = (var[i]*prbs[i]+var[j]*prbs[j])\ 
#                        /(prbs[i]+prbs[j])

# Strategy2: Eve knows the reconstruction value of each cell
        possible_yij = (var[i]+var[j])/2
        #possibel_yij = random.choice([var[i],var[j]])
        
        all_yijs = np.append(all_yijs, possible_yij)
        
        possible_dij\
        = var[i]**2*prbs[i] + possible_yij**2*prbs[i] - possible_yij*2*var[i]*prbs[i]\
        + var[j]**2*prbs[j] + possible_yij**2*prbs[j] - possible_yij*2*var[j]*prbs[j]
        all_dijs = np.append(all_dijs, possible_dij)
        
        possible_prb0 = max(prbs[j],prbs[i])
        all_prb0s = np.append(all_prb0s, possible_prb0)

# LP

In [402]:
'''For distortion and entropy, we implement the linear programming.'''
Omega = 1
Lambda = 100000000

obj_coef = -Omega * all_dijs - Lambda * all_prb0s
all_length = int(cells_num * (cells_num-1) / 2)

A_eq = np.zeros([cells_num, all_length])
k = 0
for i in range(0, cells_num-1):
    for j in range(i+1,cells_num):
        A_eq[i,k]=1
        k+=1
k = 0
for j in range(1, cells_num):
    for i in range (j,cells_num):
        A_eq[i,k]=1
        k+=1

b_eq = np.ones([cells_num,])

result = linprog(obj_coef, A_eq=A_eq, b_eq=b_eq)
opt_xs = result.x
opt_pairs = np.round(result.x) # Pairs
# opt_distortion = -result.fun # Value of the distortion

opt_dijs = opt_pairs * all_dijs
opt_distortion = sum(opt_dijs)
opt_db = 10 * np.log10(opt_distortion)

opt_prb0s = opt_pairs * all_prb0s
opt_prb0 = sum(opt_prb0s)
if opt_prb0 != 1 and opt_prb0 != 0:
    opt_entropy = -opt_prb0 * np.log2(opt_prb0)-(1-opt_prb0)*np.log2(1-opt_prb0)
else:
    opt_entropy = 0

# print(opt_distortion)
print(opt_db)
print(opt_entropy)

36.16599416618452
0.4404811919757082


In [383]:
k = 0
xx = np.zeros([cells_num, cells_num])
for i in range(0,cells_num):
    for j in range(i+1,cells_num):
        xx[i,j] = opt_pairs[k]
        k +=1
        
lp_pairs_show = np.array(np.where(xx == 1))
print(lp_pairs_show)         
# print(opt_distortion)
# print(opt_xs)
# print(opt_pairs)
# print('\n', pairs)

[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
   36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
   54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
   72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
   90  91  92  93  94  95  96  97 125 126 127 128 129 130 131 132 133 134
  135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152]
 [198 199 197 200 196 201 195 202 203 194 204 193 205 192 191 206 190 189
  188 207 187 186 185 179 178 180 182 177 183 181 184 208 176 175 174 163
  173 162 172 165 164 167 166 171 170 161 160 168 169 159 209 158 157 210
  156 155 154 211 153 212 213 214 215 216 217 218 219 220 221 222 223 224
  225 226 227 124 123 122 121 120 119 118 117 116 115 114 113 112 111 110
  105 103 109 102 106 108 104 107 228 229 230 231 232 233 234 235 236 237
  238 239 240 241 242 243 244 245 246

# LP MSB Plane

In [384]:
dual_xx = xx + np.transpose(xx)

lp_msb_plane = np.zeros([height,width])
for i in range(0,height):
    for j in range(0,width):
        true_val = intensities[i][j]
        lp_pair_val = np.array(np.where(dual_xx[true_val]==1))
        if all_counts[true_val] >= all_counts[lp_pair_val]:
            lp_msb_plane[i][j] = 0
        else:
            lp_msb_plane[i][j] = 1
            
sum(sum(lp_msb_plane))

659.0

In [385]:
# [cv2.IMWRITE_PNG_COMPRESSION, 9]

lp_msb_plane255 = lp_msb_plane*255
lp_msb_uint8 = lp_msb_plane255.astype(np.uint8)
cv2.imshow('lp_msb_plane', lp_msb_uint8)
filename = '2.4msb_255_lam1e3.webp'
cv2.imwrite(filename, lp_msb_uint8)
cv2.waitKey(1000)

# lp_msb_plane255 = lp_msb_plane
# lp_msb_uint8 = lp_msb_plane255.astype(np.uint8)
# cv2.imshow('msb_plane', lp_msb_uint8)
# filename = '1.1msb_lam1e4.webp'
# cv2.imwrite(filename, lp_msb_uint8)
# cv2.waitKey(1000)

-1

In [386]:
cv2.destroyAllWindows()

# LP Encryption

In [17]:
lp_enc_intens = np.zeros([height,width])
for i in range(0,height):
    for j in range(0,width):
        true_val = intensities[i][j]
        lp_pair_val = np.array(np.where(dual_xx[true_val]==1))
        lp_paired_vals = [true_val, lp_pair_val]
        lp_enc_intens[i][j] = random.choice(lp_paired_vals)

In [20]:
lp_uint8 = lp_enc_intens.astype(np.uint8)
cv2.imshow('lp_enc', lp_uint8)
filename = 'lp_enc.bmp'
cv2.imwrite(filename, lp_uint8)
cv2.waitKey(1000)

-1

In [21]:
cv2.destroyAllWindows()

# NBIA

In [356]:
nb_all_yijs = []
nb_all_dijs = []
power = np.log2(cells_num)
for i in range(0,int(cells_num/2)):
    j = int(cells_num/2 + i)
#     print(i,j)
    nb_yij = (var[i]+var[j])/2
    nb_all_yijs = np.append(nb_all_yijs, nb_yij)
    
    nb_dij\
        = var[i]**2*prbs[i] + nb_yij**2*prbs[i] - nb_yij*2*var[i]*prbs[i]\
        + var[j]**2*prbs[j] + nb_yij**2*prbs[j] - nb_yij*2*var[j]*prbs[j]
    nb_all_dijs = np.append(nb_all_dijs, nb_dij)
    
nb_distortion = np.sum(nb_all_dijs)
nb_db = 10 * np.log10(nb_distortion)

nb_prb0s = prbs[0: int(cells_num/2)]  
nb_prb0 = sum(nb_prb0s)
nb_entropy = -nb_prb0*np.log2(nb_prb0)-(1-nb_prb0)*np.log2(1-nb_prb0)

print('\n',nb_distortion)
print(nb_entropy)
print(nb_db)


 4096.0
0.03499629583688607
36.12359947967774


# NB MSB Plane

In [357]:
nb_msb_plane = np.zeros([height,width])
for i in range(0,height):
    for j in range(0,width):
        true_val = intensities[i][j]
        if true_val <= 127:
            nb_msb_plane[i][j] = 0 
        else:
            nb_msb_plane[i][j] = 1
sum(sum(nb_msb_plane))

1044725.0

In [358]:
nb_msb_plane255 = nb_msb_plane*255
nb_msb_uint8 = nb_msb_plane255.astype(np.uint8)
cv2.imshow('nb_msb', nb_msb_uint8)
filename = '2.4nb_255.webp'
cv2.imwrite(filename, nb_msb_uint8)
cv2.waitKey(1000)

# nb_msb_plane255 = nb_msb_plane
# nb_msb_uint8 = nb_msb_plane255.astype(np.uint8)
# cv2.imshow('nb1_msb_plane', nb_msb_uint8)
# filename = '1.1nb.webp'
# cv2.imwrite(filename, nb_msb_uint8)
# cv2.waitKey(1000)

-1

In [359]:
cv2.destroyAllWindows()

# NB Encryption

In [22]:
nb_enc_intens = np.zeros([height,width])
for i in range(0,height):
    for j in range(0,width):
        true_val = intensities[i][j]
        if true_val <= 127:
            nb_pair_val = true_val + 128 
        else:
            nb_pair_val = true_val - 128
        nb_paired_vals = [true_val, nb_pair_val]
        nb_enc_intens[i][j] = random.choice(nb_paired_vals)

In [23]:
nb_uint8 = nb_enc_intens.astype(np.uint8)
cv2.imshow('nb_enc', nb_uint8)
filename = 'nb_enc.bmp'
cv2.imwrite(filename, nb_uint8)
cv2.waitKey(1000)

-1

# MIP

In [154]:
from mip import Model, xsum, maximize, BINARY

Omega = 0
Lambda = 1
p = Omega * all_dijs + Lambda * all_prb0s
w = A_eq
c = 1
I = range(np.shape(w)[1])

m = Model('knapsack')

x = [m.add_var(var_type=BINARY) for i in I]

m.objective = maximize(xsum(p[i] * x[i] for i in I))

for j in range (0,cells_num):
    m += xsum(w[j][i] * x[i] for i in I) <= c

opt_value = m.optimize()
        
        
# mip_xx = mip_xx + np.transpose(mip_xx)
# mip_yy = np.where(mip_xx)
# mip_zz = np.append(mip_yy[0],mip_yy[1])
# mip_pairs_show = np.reshape(mip_zz,[2,cells_num])
# mip_pairs_show = mip_pairs_show[:,0:int(cells_num/2)]

# print('\n', mip_pairs_show)
'''Do not run this cell, it takes time'''

'Do not run this cell, it takes time'

In [155]:
print(m.objective_value)
selected = [i for i in I if x[i].x >= 0.99]
# print('selected items: {}'.format(selected))

mip_pairs = []
for i in I:
    mip_pair = x[i].x
    mip_pairs = np.append(mip_pairs, mip_pair)
    
k = 0
mip_xx = np.zeros([cells_num, cells_num])
for i in range(0,cells_num):
    for j in range(i+1,cells_num):
        mip_xx[i,j] = mip_pairs[k]
        k +=1
        
mip_pairs_show = np.array(np.where(mip_xx == 1))
print(mip_pairs_show) 

0.8568687438964844
[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
   36  37  38  46  47  48  49  50  52  53  54  55  56  57  58  59  60  61
   62  63  65  66  67  68  69  70  71  72  73  74  76  77  78  79  80  83
   84  85  87  88  89  90  91  92  93  95  96  97  98  99 100 101 102 103
  112 113 115 116 117 118 119 120 121 122 123 124 125 126 127 136 137 142
  143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 161 163 166
  193 196]
 [138 114 135 133  51 139 134 140 132  94 108 109 131 106 141 130  43 107
   39  45  42 105  44 104 110  41  81  75 129 111 128  64  86  40  82 162
  174 190 184 209 160 197 191 211 237 230 255 170 243 244 169 179 249 253
  217 158 223 224 178 181 189 226 180 206 227 242 183 182 165 232 218 188
  204 254 199 167 231 241 222 215 210 248 207 251 177 247 246 236 200 245
  225 240 221 213 252 233 171 212 214 238 175 219 172 229 250 176 216 185
  235 20

In [156]:
mip_pairs_01 = []
for i in I:
    mip_pairs_01 = np.append(mip_pairs_01, x[i].x)
    
mip_dijs = mip_pairs_01 * all_dijs
mip_distortion = sum(mip_dijs)
mip_db = 10 * np.log10(mip_distortion)

mip_prb0s = mip_pairs_01 * all_prb0s
mip_prb0 = sum(mip_prb0s)
mip_entropy = -mip_prb0 * np.log2(mip_prb0)-(1-mip_prb0)*np.log2(1-mip_prb0)

# print(opt_distortion)
print(mip_db)
print(mip_entropy)

34.59736948580259
0.5923809086421057


# MIP MSB plane

In [121]:
dual_mip_xx = mip_xx + np.transpose(mip_xx)

mip_msb_plane = np.zeros([height,width])
for i in range(0,height):
    for j in range(0,width):
        true_val = intensities[i][j]
        mip_pair_val = np.array(np.where(dual_mip_xx[true_val]==1))
        if all_counts[true_val] >= all_counts[mip_pair_val]:
            mip_msb_plane[i][j] = 0
        else:
            mip_msb_plane[i][j] = 1

In [ ]:
# [cv2.IMWRITE_PNG_COMPRESSION, 9]

mip_msb_plane255 = mip_msb_plane*255
mip_msb_uint8 = mip_msb_plane255.astype(np.uint8)
cv2.imshow('mip_msb_plane', mip_msb_uint8)
filename = '1mip_msb_plane255_lam10000_comp9.png'
cv2.imwrite(filename, lp_msb_uint8, [cv2.IMWRITE_PNG_COMPRESSION, 9])
cv2.waitKey(1000)

mip_msb_plane255 = mip_msb_plane
mip_msb_uint8 = mip_msb_plane255.astype(np.uint8)
cv2.imshow('mip_msb_plane', lp_msb_uint8)
filename = '1mip_msb_plane_lam10000_comp9.png'
cv2.imwrite(filename, lp_msb_uint8, [cv2.IMWRITE_PNG_COMPRESSION, 9])
cv2.waitKey(1000)

# MIP Encryption

In [147]:
dual_mip_xx = mip_xx + np.transpose(mip_xx)

mip_enc_intens = np.zeros([height,width])
for i in range(0,height):
    for j in range(0,width):
        true_val = intensities[i][j]
        pair_val = np.array(np.where(dual_mip_xx[true_val]==1))
        paired_vals = [true_val, pair_val]
        mip_enc_intens[i][j] = random.choice(paired_vals)

NameError: name 'mip_xx' is not defined

In [95]:
print(mip_pairs_show-pairs)
munique, mcounts = np.unique(mip_pairs, return_counts=True)
print(munique, mcounts)
lpunique, lpcounts = np.unique(opt_pairs, return_counts=True)
print(lpunique, lpcounts)
'''The results of linear programming was correct, then integer linear programming must be used'''

NameError: name 'pairs' is not defined